In [0]:
%config IPCompleter.greedy = True

In [1]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

0.4.0
False


In [0]:
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms

import matplotlib as plt
%matplotlib inline

import helper

In [0]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = datasets.FashionMNIST('FashionMNIST_data/', download = True, train = True, transform = transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True)

In [0]:
test_set = datasets.FashionMNIST('FashionMNIST_data/', download = True, train = False, transform = transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True)

In [0]:
class Network(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p):
        super().__init__()
        
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
            
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes ])
        
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
        self.dropout = nn.Dropout(p = drop_p)
        
        
    def forward(self, x):
        
        for linear in self.hidden_layers:
            x = F.relu(linear(x))
            x = self.dropout(x)
        
        x = self.output(x)
        
        return F.log_softmax(x, dim = 1)

In [0]:
model = Network(784, 10, [516, 256], 0.5)
#print(model)

In [0]:
optimizer = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.NLLLoss()

In [0]:
def validation(model, test_loader, criterion):
    accuracy = 0
    test_loss = 0
    for images, labels in test_loader:
        
        images = images.resize_(images.size()[0], 784)
        
        output = model.forward(images)
        ps = torch.exp(output)
        
        test_loss += criterion(output, labels).item()
        
        equality = (labels.data == ps.max(dim = 1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [0]:
epochs = 3
steps = 0
print_every = 40
for e in range(epochs):
    running_loss = 0
    model.train()
    for images, labels in train_loader:
        steps +=1
        images.resize_(images.shape[0], 784)
        
        optimizer.zero_grad()
        
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            
            with torch.no_grad():
                test_loss, accuracy = validation(model, test_loader, criterion)
                print(
                    'running_loss : {:.3f}'.format(running_loss/print_every),
                    'test_loss : {:.3f}'.format(test_loss/len(test_loader)),
                    'accuracy {:.3f}'.format(accuracy/len(test_loader))
                )
                
            running_loss = 0
            model.train()

running_loss : 1.870 test_loss : 1.235 accuracy 0.509
running_loss : 1.318 test_loss : 0.966 accuracy 0.612
running_loss : 1.167 test_loss : 0.850 accuracy 0.711
running_loss : 1.031 test_loss : 0.800 accuracy 0.687
running_loss : 0.981 test_loss : 0.752 accuracy 0.709
running_loss : 0.934 test_loss : 0.780 accuracy 0.698
running_loss : 0.818 test_loss : 0.713 accuracy 0.731
running_loss : 0.876 test_loss : 0.727 accuracy 0.728
running_loss : 0.963 test_loss : 0.745 accuracy 0.730
running_loss : 0.884 test_loss : 0.743 accuracy 0.732
running_loss : 1.064 test_loss : 0.793 accuracy 0.692
running_loss : 0.867 test_loss : 0.691 accuracy 0.735
running_loss : 0.906 test_loss : 0.716 accuracy 0.711
running_loss : 0.775 test_loss : 0.659 accuracy 0.746
running_loss : 0.915 test_loss : 0.701 accuracy 0.735
running_loss : 0.755 test_loss : 0.717 accuracy 0.703
running_loss : 0.763 test_loss : 0.661 accuracy 0.758
running_loss : 0.908 test_loss : 0.656 accuracy 0.757
running_loss : 0.705 test_lo

running_loss : 0.550 test_loss : 0.530 accuracy 0.809
running_loss : 0.833 test_loss : 0.535 accuracy 0.800
running_loss : 0.746 test_loss : 0.544 accuracy 0.805
running_loss : 0.758 test_loss : 0.528 accuracy 0.814
running_loss : 0.622 test_loss : 0.532 accuracy 0.812
running_loss : 0.604 test_loss : 0.527 accuracy 0.807
running_loss : 0.620 test_loss : 0.534 accuracy 0.814
running_loss : 0.683 test_loss : 0.525 accuracy 0.810
running_loss : 0.582 test_loss : 0.551 accuracy 0.800
running_loss : 0.764 test_loss : 0.571 accuracy 0.800
running_loss : 0.614 test_loss : 0.579 accuracy 0.795
running_loss : 0.796 test_loss : 0.558 accuracy 0.794
running_loss : 0.621 test_loss : 0.530 accuracy 0.804
running_loss : 0.647 test_loss : 0.520 accuracy 0.814
running_loss : 0.687 test_loss : 0.524 accuracy 0.813
running_loss : 0.752 test_loss : 0.515 accuracy 0.821
running_loss : 0.613 test_loss : 0.517 accuracy 0.817
running_loss : 0.565 test_loss : 0.497 accuracy 0.818
running_loss : 0.491 test_lo

running_loss : 0.719 test_loss : 0.491 accuracy 0.829
running_loss : 0.541 test_loss : 0.470 accuracy 0.835
running_loss : 0.487 test_loss : 0.491 accuracy 0.835
running_loss : 0.646 test_loss : 0.490 accuracy 0.819
running_loss : 0.594 test_loss : 0.499 accuracy 0.816
running_loss : 0.551 test_loss : 0.486 accuracy 0.827
running_loss : 0.598 test_loss : 0.488 accuracy 0.831
running_loss : 0.582 test_loss : 0.483 accuracy 0.828
running_loss : 0.535 test_loss : 0.497 accuracy 0.817
running_loss : 0.599 test_loss : 0.501 accuracy 0.820
running_loss : 0.613 test_loss : 0.552 accuracy 0.798
running_loss : 0.638 test_loss : 0.527 accuracy 0.811
running_loss : 0.770 test_loss : 0.499 accuracy 0.820
running_loss : 0.643 test_loss : 0.495 accuracy 0.830
running_loss : 0.586 test_loss : 0.499 accuracy 0.823
running_loss : 0.832 test_loss : 0.506 accuracy 0.823
running_loss : 0.722 test_loss : 0.489 accuracy 0.822
running_loss : 0.623 test_loss : 0.490 accuracy 0.822
running_loss : 0.526 test_lo

running_loss : 0.680 test_loss : 0.483 accuracy 0.828
running_loss : 0.591 test_loss : 0.493 accuracy 0.826
running_loss : 0.677 test_loss : 0.489 accuracy 0.829
running_loss : 0.525 test_loss : 0.479 accuracy 0.826
running_loss : 0.551 test_loss : 0.489 accuracy 0.827
running_loss : 0.581 test_loss : 0.476 accuracy 0.830
running_loss : 0.571 test_loss : 0.482 accuracy 0.828
running_loss : 0.726 test_loss : 0.492 accuracy 0.827
running_loss : 0.620 test_loss : 0.474 accuracy 0.833
running_loss : 0.580 test_loss : 0.458 accuracy 0.841
running_loss : 0.609 test_loss : 0.477 accuracy 0.829
running_loss : 0.592 test_loss : 0.492 accuracy 0.826
running_loss : 0.547 test_loss : 0.488 accuracy 0.826
running_loss : 0.641 test_loss : 0.490 accuracy 0.822
running_loss : 0.629 test_loss : 0.486 accuracy 0.830
running_loss : 0.630 test_loss : 0.491 accuracy 0.837
running_loss : 0.601 test_loss : 0.525 accuracy 0.791
running_loss : 0.687 test_loss : 0.491 accuracy 0.811
running_loss : 0.577 test_lo

In [0]:
checkpoint = {
                'input_size':784,
                'output_size':10,
                'hidden_layers':[each.out_features for each in model.hidden_layers],
                'state_dict':model.state_dict()
            }

In [0]:
torch.save(checkpoint, 'checkpoint.pth')